In [1]:
import tensorflow as tf 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random
import skimage
from skimage.io import imread, imshow

In [ ]:
from tensorflow import keras
'''loading model'''
root_dir = r'F:\Leuk study re-designed\ALLIDB-2 best models\High'
model_name = r'EffNetB0_mcc_loss_high.h5'
model = keras.models.load_model(os.path.join(root_dir, model_name))
last_conv_layer_name = "top_activation"
print(model.layers[-1].name)
#model.layers[-1].activation = None
model.summary()

In [31]:
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    # First, we create a model that maps the input image to the activations
    # of the last conv layer as well as the output predictions
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )

    # Then, we compute the gradient of the top predicted class for our input image
    # with respect to the activations of the last conv layer
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # This is the gradient of the output neuron (top predicted or chosen)
    # with regard to the output feature map of the last conv layer
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # This is a vector where each entry is the mean intensity of the gradient
    # over a specific feature map channel
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))

    # We multiply each channel in the feature map array
    # by "how important this channel is" with regard to the top predicted class
    # then sum all the channels to obtain the heatmap class activation
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

In [33]:
def save_and_display_gradcam(array, heatmap, orig_name, img_class, alpha):
    # Load the original image
    #img = keras.preprocessing.image.load_img(img_path)
    #img = keras.preprocessing.image.img_to_array(img)
    img = array

    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img.shape[1], img.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img
    
    #'''my addition'''
    #test_img = superimposed_img
    
    #'''my addition'''
    #superimposed_img = superimposed_img[0,:,:,:]
    
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)
    
    #'''my addition'''
    #test = np.array(superimposed_img)
    #print(type(test))
    
    # Save the superimposed image
    if img_class == 0:
        path = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\diffrent approach heatmap\all'
        orig_name = orig_name[:-4]
        orig_name = orig_name + '.png'
        superimposed_img.save(os.path.join(path, orig_name))
    else:
        path = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\diffrent approach heatmap\hem'
        orig_name = orig_name[:-4]
        orig_name = orig_name + '.png'
        superimposed_img.save(os.path.join(path, orig_name))

    # Display Grad CAM
    #display(Image(cam_path))
    #return test_img


#arr = array[0,:,:,:]
#result = save_and_display_gradcam(arr, heatmap)

In [3]:
def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    im = img[slices].astype('float32')
    return im

In [42]:
VAL_ALL_PATH = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\Test\all'
VAL_HEM_PATH = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\Test\hem'

all_list = os.listdir(VAL_ALL_PATH) 
hem_list = os.listdir(VAL_HEM_PATH)

all_list.sort()
hem_list.sort()

# for x in range(len(all_list)):
#     img = imread(os.path.join(VAL_ALL_PATH, all_list[x]))
#     cropped_img = crop_center(img, (150,150,3))
#     rescaled_cropped_img = cropped_img * (1./255.)
#     array = np.expand_dims(rescaled_cropped_img, axis=0)
#     heatmap = make_gradcam_heatmap(array, model, last_conv_layer_name)
#     save_and_display_gradcam(rescaled_cropped_img, heatmap, all_list[x], 0, 0.4)
        
# for x in range(len(hem_list)):
#     img = imread(os.path.join(VAL_HEM_PATH, hem_list[x]))
#     cropped_img = crop_center(img, (150,150,3))
#     rescaled_cropped_img = cropped_img * (1./255.)
#     array = np.expand_dims(rescaled_cropped_img, axis=0)
#     heatmap = make_gradcam_heatmap(array, model, last_conv_layer_name)
#     save_and_display_gradcam(rescaled_cropped_img, heatmap, hem_list[x], 1, 0.4)


In [ ]:
import skimage 
from skimage.io import imread, imshow
img_all = imread(r'D:\new_leuk\Leukemia_Work_Revive\Corrected\Originall data\val\all\2.bmp')
print('MAx: ', np.max(img_all))
print('MIN: ', np.min(img_all))
cropped_img_all = crop_center(img_all, (150,150,3))
print('Cropped_img MAx: ', np.max(cropped_img_all))
print('Cropped_img MIN: ', np.min(cropped_img_all))
cropped_img_all = cropped_img_all.astype('uint8')
print(cropped_img_all.dtype)
imshow(cropped_img_all)
rescaled_cropped_img_all = cropped_img_all * (1.0/255.0)
print('Rescaled_cropped_img MAx: ', np.max(rescaled_cropped_img_all))
print('Resclased_cropped_img MIN: ', np.min(rescaled_cropped_img_all))
print(type(rescaled_cropped_img_all))
print(rescaled_cropped_img_all.shape)
print(rescaled_cropped_img_all.dtype)
array = np.expand_dims(rescaled_cropped_img_all, axis=0)
print(array.dtype, array.shape)

In [ ]:
import skimage 
from skimage.io import imread, imshow
img_all = imread(r'F:\Leuk study re-designed\ALLIDB-2 best models\High\Test\all\Im003_1.tif')
print('MAx: ', np.max(img_all))
print('MIN: ', np.min(img_all))
cropped_img_all = crop_center(img_all, (150,150,3))
print('Cropped_img MAx: ', np.max(cropped_img_all))
print('Cropped_img MIN: ', np.min(cropped_img_all))
cropped_img_all = cropped_img_all.astype('uint8')
print(cropped_img_all.dtype)
imshow(cropped_img_all)
rescaled_cropped_img_all = cropped_img_all * (1.0/255.0)
print('Rescaled_cropped_img MAx: ', np.max(rescaled_cropped_img_all))
print('Resclased_cropped_img MIN: ', np.min(rescaled_cropped_img_all))
print(type(rescaled_cropped_img_all))
print(rescaled_cropped_img_all.shape)
print(rescaled_cropped_img_all.dtype)
array = np.expand_dims(rescaled_cropped_img_all, axis=0)
print(array.dtype, array.shape)

In [ ]:
from tensorflow import keras
root_dir = r'F:\Leuk study re-designed\ALLIDB-2 best models\High'
model_name = r'EffNetB0_mcc_loss_high.h5'
'''loading model'''
# model = model.load_weights(os.path.join(root_dir, model_name))
model = keras.models.load_model(os.path.join(root_dir, model_name), compile=False)
model.summary()
print(model.layers[-1].name)
last_conv_layer_name = "conv5_block16_concat"        # Foe EfficientB0
# last_conv_layer_name = "conv5_block16_concat"  #For Densenet121

In [ ]:
pip install tf-explain

In [ ]:

# Import explainer
import tf_explain
from tf_explain.core.grad_cam import GradCAM

# Instantiation of the explainer
explainer = GradCAM()

arr = array[0,:,:,:]
data = ([arr], None)
# Call to explain() method
output = explainer.explain(data, model, class_index=0, layer_name='conv5_block16_concat', colormap=cv2.COLORMAP_JET)

# output_dir = r'D:\new_leuk\Leukemia_Work_Revive\Corrected\Grad-cam heatmap'
# output_name = 'test2.png'

# Save output
#explainer.save(output, output_dir, output_name)

for x in range(len(all_list)):
    img = imread(os.path.join(VAL_ALL_PATH, all_list[x]))
    cropped_img = crop_center(img, (150,150,3))
    rescaled_cropped_img = cropped_img * (1./255.)
    data = ([rescaled_cropped_img], None)
    output = explainer.explain(data, model, class_index=0, layer_name='conv5_block16_concat', colormap=cv2.COLORMAP_JET)
    output_dir = r'F:\Leuk study re-designed\ALLIDB-2 best models\Low\corresponding hetmaps\all'
    output_name = all_list[x]
    output_name = output_name[:-4]
    output_name = output_name + '.png'
    explainer.save(output, output_dir, output_name)

print("Done")

In [ ]:
import os
import numpy as np
import skimage
from skimage.io import imshow, imread
print(len(all_list))
for x in all_list:
    single_orig_image = imread(os.path.join(VAL_ALL_PATH, x))
    single_cropped_img = crop_center(single_orig_image, (210,210,3))
    single_cropped_img_dim_extended = np.expand_dims(single_cropped_img, axis=0)
    pred_value = model.predict(single_cropped_img_dim_extended)
    print(x, pred_value)
    

# All In One

In [ ]:
import tensorflow as tf 
physical_devices = tf.config.list_physical_devices('GPU') 
tf.config.experimental.set_memory_growth(physical_devices[0], True)


import numpy as np
import tensorflow as tf
from tensorflow import keras

# Display
from IPython.display import Image, display
import matplotlib.pyplot as plt
import matplotlib.cm as cm

import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import operator
import tensorflow as tf
import random
import skimage
from skimage.io import imread, imshow


def crop_center(img, bounding):
    start = tuple(map(lambda a, da: a//2-da//2, img.shape, bounding))
    end = tuple(map(operator.add, start, bounding))
    slices = tuple(map(slice, start, end))
    im = img[slices].astype('float32')
    return im

VAL_ALL_PATH = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\Test\all'
VAL_HEM_PATH = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\Test\hem'

# VAL_ALL_PATH = r'F:\Leuk study re-designed\C-NMC best model\High\Test enhanched\all'
# VAL_HEM_PATH = r'F:\Leuk study re-designed\C-NMC best model\High\Test enhanched\hem'

all_list = os.listdir(VAL_ALL_PATH) 
hem_list = os.listdir(VAL_HEM_PATH)

all_list.sort()
hem_list.sort()


'''FIX THESE PARAMETER'''
# PATH = VAL_ALL_PATH
# LIST = all_list

PATH = VAL_HEM_PATH
LIST = hem_list

# root_dir = r'F:\Leuk study re-designed\ALLIDB-2 best models\High'   # ALLIDB-2 High root
root_dir = r'F:\Leuk study re-designed\ALLIDB-2 best models\Low'    # ALLIDB-2 Low root

# root_dir = r'F:\Leuk study re-designed\C-NMC best model\High'       # C-NMC High root
# root_dir = r'F:\Leuk study re-designed\C-NMC best model\Low'        # C-NMC Low root

# model_name = r'HRD_EfficientNetB0_MccLoss_test1.h5'    # C-NMC High model
# model_name = r'LRD_DesnseNet121_weighted_test1.h5'     # C-NMC Low model

# model_name = r'EffNetB0_mcc_loss_high.h5'              # ALLIDB-2 High model
model_name = r'DenseNet121_low_class_weight.h5'        # ALLIDB-2 Low model

# last_conv_layer_name = "top_activation"        # For EfficientNetBo
last_conv_layer_name = "conv5_block16_concat"    #For densenet121

out_dir = r'F:\Leuk study re-designed\ALLIDB-2 best models\Low\corresponding hetmaps\hem'
index = 0  #Index of ouput layer    0 = all,,   1 = hem

crop_height = 150
crop_width = 150
'''END'''



import os
import skimage 
from skimage.io import imread, imshow
img_all = imread(os.path.join(PATH, LIST[0]))
# img_all = imread(r'F:\Leuk study re-designed\C-NMC best model\High\Test enhanched\hem\4.bmp')
print('MAx: ', np.max(img_all))
print('MIN: ', np.min(img_all))
cropped_img_all = crop_center(img_all, (crop_height,crop_width,3))
print('Cropped_img MAx: ', np.max(cropped_img_all))
print('Cropped_img MIN: ', np.min(cropped_img_all))
cropped_img_all = cropped_img_all.astype('uint8')
print(cropped_img_all.dtype)
imshow(cropped_img_all)
rescaled_cropped_img_all = cropped_img_all * (1.0/255.0)
print('Rescaled_cropped_img MAx: ', np.max(rescaled_cropped_img_all))
print('Resclased_cropped_img MIN: ', np.min(rescaled_cropped_img_all))
print(type(rescaled_cropped_img_all))
print(rescaled_cropped_img_all.shape)
print(rescaled_cropped_img_all.dtype)
array = np.expand_dims(rescaled_cropped_img_all, axis=0)
print(array.dtype, array.shape)


from tensorflow import keras
# root_dir = r'F:\Leuk study re-designed\ALLIDB-2 best models\High'
# model_name = r'EffNetB0_mcc_loss_high.h5'
'''loading model'''
# model = model.load_weights(os.path.join(root_dir, model_name))
model = keras.models.load_model(os.path.join(root_dir, model_name), compile=False)
model.summary()
print(model.layers[-1].name)
# last_conv_layer_name = "conv5_block16_concat"        # Foe EfficientB0
# last_conv_layer_name = "conv5_block16_concat"  #For Densenet121


#pip install tf-explain





# Import explainer
import tf_explain
from tf_explain.core.grad_cam import GradCAM

# Instantiation of the explainer
explainer = GradCAM()

arr = array[0,:,:,:]
data = ([arr], None)
# Call to explain() method
output = explainer.explain(data, model, class_index=index, layer_name=last_conv_layer_name, colormap=cv2.COLORMAP_JET)

# output_dir = r'D:\new_leuk\Leukemia_Work_Revive\Corrected\Grad-cam heatmap'
# output_name = 'test2.png'

# Save output
#explainer.save(output, output_dir, output_name)

for x in range(len(LIST)):
    img = imread(os.path.join(PATH, LIST[x]))
    cropped_img = crop_center(img, (crop_height,crop_width,3))
    rescaled_cropped_img = cropped_img * (1./255.)
    data = ([rescaled_cropped_img], None)
    output = explainer.explain(data, model, class_index=index, layer_name=last_conv_layer_name, colormap=cv2.COLORMAP_JET)
    output_dir = out_dir
    output_name = LIST[x]
    output_name = output_name[:-4]
    output_name = output_name + '.png'
    explainer.save(output, output_dir, output_name)

print("Done")




import os
import numpy as np
import skimage
from skimage.io import imshow, imread, imsave
print(len(LIST))
pred_list = []
for x in LIST:
    single_orig_image = imread(os.path.join(PATH, x))
    single_cropped_img = crop_center(single_orig_image, (crop_height,crop_width,3))
    single_cropped_img_dim_extended = np.expand_dims(single_cropped_img, axis=0)
    single_cropped_img_dim_extended = single_cropped_img_dim_extended / 255.0
    pred_value = model.predict(single_cropped_img_dim_extended)
    print(x, pred_value)
    if pred_value > 0.5:
        pred_list.append(1)
        pred_flat = 1
    else:
        pred_list.append(0)
        pred_flat = 0
    des_path = out_dir + '/' + x[:-4] + '_' + str(pred_flat) + '.png'
    imsave(des_path, single_cropped_img)
    

print('Number of 0 :', pred_list.count(0))
print('Number of 1 :', pred_list.count(1))

In [ ]:
l = '1037_1.png'
l[:-6]

In [ ]:
import os

path = r'F:\Leuk study re-designed\C-NMC best model\High\corresponding heatmaps\all'
all_images = os.listdir(path)
all_images.sort()
all_images
#''
target_path0 = r'F:\Leuk study re-designed\C-NMC best model\High\corresponding heatmaps\all\classsified'
target_path1 = r'F:\Leuk study re-designed\C-NMC best model\High\corresponding heatmaps\all\misclasssified'

for x in range (len(all_images)):
    if all_images[x][-5:-4] == 0:
        orig = imread(os.path.join(path, all_images[x]))
        final_path = target_path0 + '/' + all_images[x] 
        imsave(final_path, orig)
        print(all_images[x][:-6])
        name = all_images[x][:-6] + '.png'
        print('name: ', name)
        heatmap = imread(os.path.join(path, name))
        final_path = target_path0 + '/' + all_images[x][:-6] + '.png'
        imsave(final_path, heatmap)
    else:
        orig = imread(os.path.join(path, all_images[x]))
        final_path = target_path1 + '/' + all_images[x] 
        imsave(final_path, orig)
        print(all_images[x])
        print(all_images[x][:-6])
        name = all_images[x][:-6] + '.png'
        print('name: ', name)
        heatmap = imread(os.path.join(path, name))
        final_path = target_path1 + '/' + all_images[x][:-6] + '.png'
        imsave(final_path, heatmap)
#''

In [ ]:
a = '134sdfdfd.png'
b = '_0.png'
print(a[:-4])
print(b[-5:-4])

In [ ]:
import os

path = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\corresponding hetmaps\all'
all_images = os.listdir(path)
# all_images.sort()
# print(len(all_images)/2)

heatmap = []
orig = []
for x in all_images:
    if '_' not in x:
        heatmap.append(x)
    else:
        orig.append(x)
print(len(heatmap))
print(len(orig))

target_path0 = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\corresponding hetmaps\all classified'
target_path1 = r'F:\Leuk study re-designed\ALLIDB-2 best models\High\corresponding hetmaps\all misclassified'


for x in heatmap:
    for y in orig:
        if x[:-4] == y[:-6]:  
            print(x, y)
#             print(y[-5:-4])
            if y[-5:-4] == '0':
                print('Classified')
                htmap = imread(os.path.join(path, x))
                final_path = target_path0 + '/' + x
                print(final_path)
                imsave(final_path, htmap)
                
                original = imread(os.path.join(path, y))
                final_path = target_path0 + r'/' + y 
                imsave(final_path, original)
            elif y[-5:-4] == '1':
                print('misclassified')
                htmap = imread(os.path.join(path, x))
                final_path = target_path1 + r'/' + x
                print(final_path)
                imsave(final_path, htmap)
                
                original = imread(os.path.join(path, y))
                final_path = target_path1 + r'/' + y 
                imsave(final_path, original)